In [1]:
from copy import deepcopy
from IPython.display import clear_output
import time

walls = set()
boxes = set()
robot = None
moves = ''

directions = {
    '<':(0,-1),
    '>':(0,1),
    '^':(-1,0),
    'v':(1,0)
}

filename = 'input.txt'
with open(filename, 'r') as fl:
    first = True
    for i, ln in enumerate(fl):
        if ln.strip() == '':
            first = False
        if first:
            for j, s in enumerate(ln.strip()):
                if s == '#':
                    walls.add((i,j))
                elif s == 'O':
                    boxes.add((i,j))
                elif s == '@':
                    robot = (i,j)
        else:
            moves += ln.strip()
            
n, m = max(w[0] for w in walls) + 1, max(w[1] for w in walls) + 1
init_boxes = deepcopy(boxes)
init_robot = deepcopy(robot)

In [2]:
def print_warehouse(walls, boxes, robot, n, m, wide=False):
    s = ''
    for i in range(n):
        for j in range(m):
            if (i,j) in walls:
                s += '#'
            elif not wide and (i,j) in boxes:
                s += 'O'
            elif wide and (i,j) in boxes:
                s += '['
            elif wide and (i,j-1) in boxes:
                s += ']'
            elif (i,j) == robot:
                s += '@'
            else:
                s += '.'
        s += '\n'
    return s
#print(print_warehouse(walls, boxes, robot, n, m))

In [3]:
movie=False

In [4]:
boxes = deepcopy(init_boxes)
robot = deepcopy(init_robot)
for ii, d in enumerate(moves):
    can_move = False
    # look into moving the robot
    di, dj = directions[d]
    new_bot = robot[0] + di, robot[1] + dj
    box_moves = {}
    if new_bot not in walls:
        # now look into boxes:
        if new_bot in boxes:
            box = new_bot
            while True:
                new_box = box[0] + di, box[1] + dj
                if new_box in walls:
                    # can't move the box, so we're done
                    break 
                elif new_box in boxes:
                    # we hit another box, so log this move and 
                    # keep checking
                    box_moves[box] = new_box
                    box = new_box
                else:
                    # we hit nothing, so log this move and finish
                    box_moves[box] = new_box
                    can_move = True
                    break
        else:
            can_move = True
            
    if can_move:
        robot = new_bot
        for old_box in box_moves.keys():
            boxes.remove(old_box)
        for new_box in box_moves.values():
            boxes.add(new_box)
    if movie:
        clear_output(wait=True)
        print('move',ii,'of',len(moves), '---', d)
        print(print_warehouse(walls, boxes, robot, n, m))
        time.sleep(0.01)
        

In [5]:
print(sum(i*100 + j for i,j in boxes))

1463512


In [6]:
# part 2
walls = set()
boxes = set()
robot = None

directions = {
    '<':(0,-1),
    '>':(0,1),
    '^':(-1,0),
    'v':(1,0)
}

with open(filename, 'r') as fl:
    for i, ln in enumerate(fl):
        if ln.strip() == '':
            break
        for j, s in enumerate(ln.strip()):
            if s == '#':
                walls.add((i,j*2))
                walls.add((i,j*2+1))
            elif s == 'O':
                boxes.add((i,j*2))
            elif s == '@':
                robot = (i,j*2)
            
n, m = max(w[0] for w in walls) + 1, max(w[1] for w in walls) + 1
init_boxes = deepcopy(boxes)
init_robot = deepcopy(robot)

In [7]:
#print(print_warehouse(walls, boxes, robot, n, m, wide=True))

In [8]:
movie=False

In [9]:
boxes = deepcopy(init_boxes)
robot = deepcopy(init_robot)
for ii, d in enumerate(moves):
    can_move = False
    # look into moving the robot
    di, dj = directions[d]
    new_bot = robot[0] + di, robot[1] + dj
    box_moves = {}
    if new_bot not in walls:
        # now look into boxes accounting for the double wide
        if new_bot in boxes:
            new_boxes = (new_bot,)
        elif (new_bot[0], new_bot[1]-1) in boxes:
            new_boxes = ((new_bot[0], new_bot[1]-1),)
        else:
            new_boxes = False
            can_move = True
        # we hit a box, so figure out these moves    
        while new_boxes:
            next_boxes = set()
            for box in new_boxes:
                new_box = box[0] + di, box[1] + dj
                fatty = new_box[0], new_box[1] + 1
                other_fatty = new_box[0], new_box[1] - 1
                if new_box in walls or fatty in walls:
                    # can't move the box, so we're done
                    next_boxes = False
                    break 
                    
                box_moves[box] = new_box
                if d in ('^','v') and new_box in boxes:
                    # we hit another box left corner to left corner
                    # keep checking
                    next_boxes.add(new_box)
                if d != '<' and fatty in boxes:
                    # we hit another box right corner to left corner
                    next_boxes.add(fatty)
                if d != '>' and other_fatty in boxes:
                    # we hit a box left corner to right corner
                    next_boxes.add(other_fatty)

            new_boxes = next_boxes
        
        if new_boxes is not False:
            can_move = True
            
    if can_move:
        robot = new_bot
        for old_box in box_moves.keys():
            boxes.remove(old_box)
        for new_box in box_moves.values():
            boxes.add(new_box)
            
    if movie:
        clear_output(wait=True)
        print('move',ii,'of',len(moves), '---', d)
        print(print_warehouse(walls, boxes, robot, n, m, wide=True))
        time.sleep(0.005)

In [10]:
print(sum(i*100 + j for i,j in boxes))

1486520
